<a href="https://colab.research.google.com/github/520817/gp/blob/main/adhd(ner%20%EB%AA%A8%EB%8D%B8%20%EC%B5%9C%EC%A2%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(42)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [5]:
!git clone https://github.com/kmounlp/NER.git

fatal: destination path 'NER' already exists and is not an empty directory.


In [6]:
import os
import glob

In [7]:
file_list=[]

In [8]:
for x in os.walk('NER/'):
  for y in glob.glob(os.path.join(x[0], '*_NER.txt')):
    file_list.append(y)

In [9]:
file_list = sorted(file_list)

In [10]:
from pathlib import Path

file_path = file_list[0]
file_path = Path(file_path)
raw_text = file_path.read_text().strip()

In [11]:
!pip install korpora

from Korpora import Korpora
corpus = Korpora.load("naver_changwon_ner")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : 네이버 + 창원대
    Repository : https://github.com/naver/nlp-challenge/tree/master/missions/ner
    References : http://air.changwon.ac.kr/?page_id=10

    개체명(Named Entity)은 인명, 기관명, 지명 등과 같이 문장 또는 문서에서 특정한 의미를 가지고 있는 단어 또는 어구를 말합니다.
    이 때문에 개체명은 정보 검색 및 언어 이해를 위한 분석에서 주요한 대상으로 다루어지고 있습니다.
    Data.ly에서는 개체명 코퍼스를 제공하여 연구에 도움을 드리고자 하며, 공개적인 리더보드를 통해 많은 분들의 연구 동향을 논의/공유하고자 합니다.
    제공되는 코퍼스는 Data.ly에서 제작한 것으로, 연구 및 리더보드를 위한 학습으로 사용 가능하며 상업적인 목적으로 사용될 수 없습니다.

    # License
    연구 및 리더보드를 위한 학습으로 사용 가능하며 상업적인 목적으로 사용될 수 없습니다.

[Korpora] Corpus `naver_changwon_ner` is already installed at /root/Korpora/naver_changwon_ner/train_data


네이버 데이터셋 전처리

In [12]:
def naver_read_file(file_list):

    token_docs = []
    tag_docs = []

    for doc in file_list:
        tokens = []
        tags = []
        list1=doc.words
        list2=doc.tags

        for text,tag in zip(list1,list2):
            for i, syllable in enumerate(text): # 음절 단위로 자르고
                tokens.append(syllable)
                if tag == '-':
                  tag = 'O' # 태그가 '-'인 경우 'O'로 변경
                tags.append(tag)

        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [13]:
naver_text,naver_tags =naver_read_file(corpus.train)

In [14]:
print(len(naver_text))
print(len(naver_tags))

90000
90000


In [15]:
print(naver_text[0], end='\n\n') # 음절 단위로 잘 잘렸네요!
print(naver_tags[0])

['비', '토', '리', '오', '양', '일', '만', '에', '영', '사', '관', '감', '호', '용', '퇴', ',', '항', '룡', '압', '력', '설', '의', '심', '만', '가', '율']

['PER_B', 'PER_B', 'PER_B', 'PER_B', 'DAT_B', 'DAT_B', 'O', 'O', 'ORG_B', 'ORG_B', 'ORG_B', 'CVL_B', 'CVL_B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


해양 뭐시기 데이터셋 전처리

In [16]:
import re
def read_file(file_list):
    token_docs = []
    tag_docs = []
    for file_path in file_list:
        #print("read file from ", file_path)
        file_path = Path(file_path)
        raw_text = file_path.read_text().strip()
        raw_docs = re.split(r'\n\t?\n', raw_text)

        for doc in raw_docs:
            tokens = []
            tags = []
            for line in doc.split('\n'):
                if line[0:1] == "$" or line[0:1] == ";" or line[0:2] == "##":
                    continue
                token = line.split('\t')[0]
                tag = line.split('\t')[3]
                for i, syllable in enumerate(token):    # 음절 단위로 잘라서
                    tokens.append(syllable)
                    tags.append(tag)

            token_docs.append(tokens)
            tag_docs.append(tags)

    return token_docs, tag_docs

In [17]:
texts, tags = read_file(file_list[:])

In [18]:
print(len(texts))
print(len(tags))

19263
19263


In [19]:
print(texts[0], end='\n\n') # 음절 단위로 잘 잘렸네요!
print(tags[0])

['오', '에', '_', '겐', '자', '부', '로', '는', '_', '일', '본', '_', '현', '대', '문', '학', '의', '_', '초', '석', '을', '_', '놓', '은', '_', '것', '으', '로', '_', '평', '가', '받', '는', '_', '작', '가', '_', '나', '쓰', '메', '_', '소', '세', '키', '(', '1', '8', '6', '7', '~', '1', '9', '1', '6', ')', '의', '_', '대', '표', '작', '_', '‘', '마', '음', '’', '에', '_', '담', '긴', '_', '군', '국', '주', '의', '적', '_', '요', '소', ',', '_', '야', '스', '쿠', '니', '_', '신', '사', '_', '참', '배', '_', '행', '위', '까', '지', '_', '소', '설', '의', '_', '삽', '화', '로', '_', '동', '원', '하', '며', '_', '일', '본', '_', '사', '회', '의', '_', '‘', '비', '정', '상', '성', '’', '을', '_', '문', '제', '_', '삼', '는', '다', '.']

['B-PER', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'B-PER', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'B-DUR', 'B-DUR', 'B-DUR', 'B-DUR', 'I-DUR', 'I-DUR',

분리된 토큰을 하나의 개체명으로 묶어 주는 태깅 시스템을 위해 있는게 BIO, BIESO임.

여러 개의 토큰으로 이루어진 개체명의 경우

**개체명이 시작할 때** 'B,begin'

**토큰이 개체명 중간에 있을 때** 'I,inside'

**개체명의 마지막에 위치할 때** 'E,end'

**하나의 토큰이 곧 하나의 개체명일 때** 'S,singleton'

**토큰이 개체명이 아닐 경우** 'O,outside'

In [20]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

for i, tag in enumerate(unique_tags):
    print(tag)  #해양 tags 변경 전

I-ORG
B-PNT
B-DUR
I-PNT
I-MNY
I-PER
B-DAT
O
B-PER
I-NOH
B-NOH
B-LOC
I-TIM
B-MNY
I-DAT
B-TIM
I-DUR
I-POH
B-ORG
B-POH
I-LOC


In [21]:
for sublist in tags:
    for i, label in enumerate(sublist):
        if label == 'I-NOH':
            sublist[i] = 'NOH_I'
        elif label == 'I-PNT':
            sublist[i] = 'PNT_I'
        elif label == 'I-ORG':
            sublist[i] = 'ORG_I'
        elif label == 'I-DUR':
            sublist[i] = 'DUR_I'
        elif label == 'I-TIM':
            sublist[i] = 'TIM_I'
        elif label == 'I-PER':
            sublist[i] = 'PER_I'
        elif label == 'I-MNY':
            sublist[i] = 'MNY_I'
        elif label == 'B-MNY':
            sublist[i] = 'MNY_B'
        elif label == 'B-DAY':
            sublist[i] = 'DAY_B'
        elif label == 'B-TIM':
            sublist[i] = 'TIM_B'
        elif label == 'I-LOC':
            sublist[i] = 'LOC_I'
        elif label == 'B-PER':
            sublist[i] = 'PER_B'
        elif label == 'B-ORG':
            sublist[i] = 'ORG_B'
        elif label == 'B-LOC':
            sublist[i] = 'LOC_B'
        elif label == 'B-PNT':
            sublist[i] = 'PNT_B'
        elif label == 'B-NOH':
            sublist[i] = 'NOH_B'
        elif label == 'I-PER':
            sublist[i] = 'PER_I'
        elif label == 'B-LOC':
            sublist[i] = 'LOC_B'
        elif label == 'I-POH':
            sublist[i] = 'POH_I'
        elif label == 'I-DAT':
            sublist[i] = 'DAT_I'
        elif label == 'B-POH':
            sublist[i] = 'POH_B'
        elif label == 'B-DUR':
            sublist[i] = 'DUR_B'
        elif label == 'B-DAT':
            sublist[i] = 'DAT_B'
        elif label == '0':
            sublist[i] = '0'

In [22]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

for i, tag in enumerate(unique_tags):
    print(tag)  #해양 tags 변경 후

POH_B
TIM_I
PER_I
POH_I
TIM_B
O
DAT_I
DUR_I
PNT_B
PNT_I
MNY_I
ORG_I
MNY_B
ORG_B
LOC_B
DAT_B
DUR_B
PER_B
LOC_I
NOH_B
NOH_I


In [23]:
texts.extend(naver_text)
tags.extend(naver_tags)

In [24]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [25]:
for i, tag in enumerate(unique_tags):
    print(tag)  # 학습을 위한 label list를 확인합니다.

POH_B
CVL_B
FLD_B
FLD_I
EVT_I
AFW_B
CVL_I
NUM_B
MAT_I
TIM_I
ANM_I
PER_I
POH_I
TRM_I
TIM_B
ANM_B
O
DAT_I
DUR_I
PNT_B
PNT_I
MNY_I
EVT_B
ORG_I
MNY_B
ORG_B
LOC_B
PLT_I
MAT_B
DAT_B
NUM_I
AFW_I
PLT_B
DUR_B
PER_B
LOC_I
NOH_B
TRM_B
NOH_I


In [26]:
for tag in list(tag2id.keys()) :
    globals()[tag] = 0

for tag in tags :
    for ner in tag :
        globals()[ner] += 1

for tag in list(tag2id.keys()) :
    print('{:>6} : {:>7,}'. format(tag, globals()[tag]))

 POH_B :  18,025
 CVL_B : 266,517
 FLD_B :   9,715
 FLD_I :     155
 EVT_I :  31,340
 AFW_B :  23,535
 CVL_I :  15,293
 NUM_B : 228,990
 MAT_I :      62
 TIM_I :   5,195
 ANM_I :     172
 PER_I :  32,070
 POH_I :  25,817
 TRM_I :  14,924
 TIM_B :  11,147
 ANM_B :  22,070
     O : 3,067,701
 DAT_I :  29,419
 DUR_I :   3,549
 PNT_B :   4,430
 PNT_I :   1,855
 MNY_I :   4,929
 EVT_B :  59,714
 ORG_I :  45,175
 MNY_B :   3,441
 ORG_B : 279,040
 LOC_B : 108,316
 PLT_I :       5
 MAT_B :   1,001
 DAT_B :  90,643
 NUM_I :  26,913
 AFW_I :   9,433
 PLT_B :     952
 DUR_B :   2,231
 PER_B : 213,186
 LOC_I :   7,656
 NOH_B :  18,839
 TRM_B : 129,695
 NOH_I :  16,179


In [27]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_tags, test_tags = train_test_split(texts, tags, test_size=.2, random_state=42)

In [28]:
print('Train 문장 : {:>6,}' .format(len(train_texts)))
print('Train 태그 : {:>6,}' .format(len(train_tags)))
print('Test  문장 : {:>6,}' .format(len(test_texts)))
print('Test  태그 : {:>6,}' .format(len(test_tags)))

Train 문장 : 87,410
Train 태그 : 87,410
Test  문장 : 21,853
Test  태그 : 21,853


In [29]:
train_texts[5]

['첫',
 '출',
 '발',
 '남',
 '상',
 '은',
 '작',
 '은',
 '어',
 '간',
 '하',
 '나',
 '였',
 '어',
 '요',
 '.']

BERT 토크나이저

In [30]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [58]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [32]:
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

In [33]:
def ner_tokenizer(sent, max_seq_length):
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1

    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [34]:
print(ner_tokenizer(train_texts[0], 5))

{'input_ids': [2, 1912, 4044, 4040, 3], 'attention_mask': [1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0]}


In [35]:
tokenized_train_sentences = []
tokenized_test_sentences = []

for text in train_texts:    # 전체 데이터를 tokenizing 합니다.
    tokenized_train_sentences.append(ner_tokenizer(text, 128))
for text in test_texts:
    tokenized_test_sentences.append(ner_tokenizer(text, 128))

In [36]:
def encode_tags(tags, max_seq_length):
    # label 역시 입력 token과 개수를 맞춰줍니다
    tags = tags[:max_seq_length-2]
    labels = [tag2id[tag] for tag in tags]
    labels = [tag2id['O']] + labels

    padding_length = max_seq_length - len(labels)
    labels = labels + ([pad_token_label_id] * padding_length)

    return labels

In [37]:
tag2id

{'POH_B': 0,
 'CVL_B': 1,
 'FLD_B': 2,
 'FLD_I': 3,
 'EVT_I': 4,
 'AFW_B': 5,
 'CVL_I': 6,
 'NUM_B': 7,
 'MAT_I': 8,
 'TIM_I': 9,
 'ANM_I': 10,
 'PER_I': 11,
 'POH_I': 12,
 'TRM_I': 13,
 'TIM_B': 14,
 'ANM_B': 15,
 'O': 16,
 'DAT_I': 17,
 'DUR_I': 18,
 'PNT_B': 19,
 'PNT_I': 20,
 'MNY_I': 21,
 'EVT_B': 22,
 'ORG_I': 23,
 'MNY_B': 24,
 'ORG_B': 25,
 'LOC_B': 26,
 'PLT_I': 27,
 'MAT_B': 28,
 'DAT_B': 29,
 'NUM_I': 30,
 'AFW_I': 31,
 'PLT_B': 32,
 'DUR_B': 33,
 'PER_B': 34,
 'LOC_I': 35,
 'NOH_B': 36,
 'TRM_B': 37,
 'NOH_I': 38}

In [38]:
encode_tags(train_tags[0], 5)

[16, 34, 34, 34, 16]

In [39]:
train_labels = []
test_labels = []

for tag in train_tags:
    train_labels.append(encode_tags(tag, 128))

for tag in test_tags:
    test_labels.append(encode_tags(tag, 128))

In [40]:
len(train_labels), len(test_labels)

(87410, 21853)

Token 데이터셋

In [41]:
import torch

class TokenDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.encodings[idx].items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TokenDataset(tokenized_train_sentences, train_labels)
test_dataset = TokenDataset(tokenized_test_sentences, test_labels)

In [43]:
#!pip install transformers==4.30

In [44]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments, AutoModelForTokenClassification,EarlyStoppingCallback
import sys
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000, # 1000번쨰 steps마다 log를 보여줌
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=5,
    save_strategy='steps', # steps로 해야 earlystop이 가능
    evaluation_strategy='steps',
    save_steps=1000, # 1000번쨰 step마다 저장
    eval_steps=1000, # 1000번째 step마다 평가
    seed=15,
    load_best_model_at_end=True # 가장 좋은 성능의 모델로...
)

BertForTokenClassification

In [45]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_tags))
model.to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,# evaluation dataset
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)] #loss가 2번 감소하지 않으면 스탑
)

pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.weight', 'classifier.bias']

In [46]:
import gc
gc.collect()

108

In [48]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [49]:
trainer.evaluate()

Step,Training Loss,Validation Loss
1000,0.347600,0.229978
2000,0.215900,0.182514
3000,0.182000,0.169995
4000,0.171000,0.155971
5000,0.161600,0.148856
6000,0.148200,0.138977
7000,0.148600,0.134349
8000,0.143800,0.131404
9000,0.142300,0.127040
10000,0.134500,0.126455


Step,Training Loss,Validation Loss
22,No log,0.111767


{'eval_loss': 0.11176680028438568}

In [50]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

Step,Training Loss,Validation Loss
22,No log,0.111767


(21853, 128, 39) (21853, 128)


In [51]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [52]:
index_to_ner = {i:j for j, i in tag2id.items()}
f_label = [i for i, j in tag2id.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(predictions.label_ids).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(preds).astype(int).tolist()]

In [53]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [54]:
#데이터셋 추가
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

              precision    recall  f1-score   support

       POH_B       0.63      0.58      0.60      3333
       CVL_B       0.78      0.81      0.79     51705
       FLD_B       0.60      0.44      0.51      2046
       FLD_I       0.00      0.00      0.00        29
       EVT_I       0.70      0.74      0.72      6233
       AFW_B       0.71      0.43      0.53      4589
       CVL_I       0.45      0.34      0.39      2798
       NUM_B       0.93      0.93      0.93     45374
       MAT_I       0.00      0.00      0.00        26
       TIM_I       0.84      0.85      0.84       959
       ANM_I       0.00      0.00      0.00        30
       PER_I       0.69      0.68      0.69      6533
       POH_I       0.71      0.62      0.66      5117
       TRM_I       0.46      0.38      0.42      2574
       TIM_B       0.81      0.80      0.81      2117
       ANM_B       0.74      0.68      0.71      4497
           O       0.99      0.99      0.99   2450478
       DAT_I       0.83    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
trainer.save_model('kcelectra_base_new')

In [63]:
id2tag.values()

dict_values(['POH_B', 'CVL_B', 'FLD_B', 'FLD_I', 'EVT_I', 'AFW_B', 'CVL_I', 'NUM_B', 'MAT_I', 'TIM_I', 'ANM_I', 'PER_I', 'POH_I', 'TRM_I', 'TIM_B', 'ANM_B', 'O', 'DAT_I', 'DUR_I', 'PNT_B', 'PNT_I', 'MNY_I', 'EVT_B', 'ORG_I', 'MNY_B', 'ORG_B', 'LOC_B', 'PLT_I', 'MAT_B', 'DAT_B', 'NUM_I', 'AFW_I', 'PLT_B', 'DUR_B', 'PER_B', 'LOC_I', 'NOH_B', 'TRM_B', 'NOH_I'])

In [64]:
# 저장한 모델 불러오기
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tag2id = {'POH_B': 0, 'CVL_B': 1, 'FLD_B': 2, 'FLD_I': 3, 'EVT_I': 4,'AFW_B': 5,'CVL_I': 6,'NUM_B': 7,'MAT_I': 8, 'TIM_I': 9,'ANM_I': 10,'PER_I': 11,'POH_I': 12,
 'TRM_I': 13,
 'TIM_B': 14,
 'ANM_B': 15,
 'O': 16,
 'DAT_I': 17,
 'DUR_I': 18,
 'PNT_B': 19,
 'PNT_I': 20,
 'MNY_I': 21,
 'EVT_B': 22,
 'ORG_I': 23,
 'MNY_B': 24,
 'ORG_B': 25,
 'LOC_B': 26,
 'PLT_I': 27,
 'MAT_B': 28,
 'DAT_B': 29,
 'NUM_I': 30,
 'AFW_I': 31,
 'PLT_B': 32,
 'DUR_B': 33,
 'PER_B': 34,
 'LOC_I': 35,
 'NOH_B': 36,
 'TRM_B': 37,
 'NOH_I': 38}
unique_tags={'POH_B', 'CVL_B', 'FLD_B', 'FLD_I', 'EVT_I', 'AFW_B', 'CVL_I', 'NUM_B', 'MAT_I', 'TIM_I', 'ANM_I', 'PER_I', 'POH_I', 'TRM_I', 'TIM_B', 'ANM_B', 'O', 'DAT_I', 'DUR_I', 'PNT_B', 'PNT_I', 'MNY_I', 'EVT_B', 'ORG_I', 'MNY_B', 'ORG_B', 'LOC_B', 'PLT_I', 'MAT_B', 'DAT_B', 'NUM_I', 'AFW_I', 'PLT_B', 'DUR_B', 'PER_B', 'LOC_I', 'NOH_B', 'TRM_B', 'NOH_I'}
id2tag={0: 'POH_B',
 1: 'CVL_B',
 2: 'FLD_B',
 3: 'FLD_I',
 4: 'EVT_I',
 5: 'AFW_B',
 6: 'CVL_I',
 7: 'NUM_B',
 8: 'MAT_I',
 9: 'TIM_I',
 10: 'ANM_I',
 11: 'PER_I',
 12: 'POH_I',
 13: 'TRM_I',
 14: 'TIM_B',
 15: 'ANM_B',
 16: 'O',
 17: 'DAT_I',
 18: 'DUR_I',
 19: 'PNT_B',
 20: 'PNT_I',
 21: 'MNY_I',
 22: 'EVT_B',
 23: 'ORG_I',
 24: 'MNY_B',
 25: 'ORG_B',
 26: 'LOC_B',
 27: 'PLT_I',
 28: 'MAT_B',
 29: 'DAT_B',
 30: 'NUM_I',
 31: 'AFW_I',
 32: 'PLT_B',
 33: 'DUR_B',
 34: 'PER_B',
 35: 'LOC_I',
 36: 'NOH_B',
 37: 'TRM_B',
 38: 'NOH_I'}
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

model = AutoModelForTokenClassification.from_pretrained('kcelectra_base_new', num_labels=len(unique_tags))
model.to(device)

ElectraForTokenClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [65]:
# 기존 토크나이저는 wordPiece tokenizer로 tokenizing 결과를 반환합니다.
# 데이터 단위를 음절 단위로 변경했기 때문에, tokenizer도 음절 tokenizer로 변경

# berttokenizer를 사용하는데 한국어 vocab이 8000개 정도 밖에 없고 그 안의 한국어들의 거의 음절로 존재
# -> 음절 단위 tokenizer를 적용하면 vocab id를 어느 정도 획득할 수 있어 UNK가 별로 없을듯 하다
def ner_tokenizer(sent, max_seq_length):
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1

    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [66]:
def ner_inference(text) :

    model.eval()
    text = text.replace(' ', '_')

    predictions , true_labels = [], []

    tokenized_sent = ner_tokenizer(text, len(text)+2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    print('{}\t{}'.format("TOKEN", "TAG"))
    print("===========")
    # for token, tag in zip(tokenizer.decode(tokenized_sent['input_ids']), pred_tags):
    #   print("{:^5}\t{:^5}".format(token, tag))
    for i, tag in enumerate(pred_tags):
        print("{:^5}\t{:^5}".format(tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]), tag))

In [ ]:
text='모레 10시 반 안과 방문 예정, 시력 물어보기'

In [76]:
ner_inference(text)

TOKEN	TAG
[CLS]	  O  
  1  	DUR_B
 ##2 	DUR_B
[UNK]	DUR_I
 ##2 	DUR_I
 ##0 	DUR_I
[UNK]	DUR_I
 ##1 	DUR_I
 ##2 	DUR_I
[UNK]	DUR_I
 ##2 	DUR_I
 ##5 	DUR_I
 ##일 	DUR_I
 ##은 	  O  
  _  	  O  
  크  	POH_B
 ##리 	POH_B
 ##스 	POH_B
 ##마 	POH_B
 ##스 	POH_B
  _  	POH_I
  파  	POH_I
 ##티 	  O  
 ##를 	  O  
  _  	  O  
  준  	  O  
 ##비 	  O  
 ##할 	  O  
 ##거 	  O  
 ##야 	  O  
[SEP]	  O  
